###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# This notebook outlines the download and formatting process for the Center for Neighborhood Technology's Housing and Transportation Cost Index as well as the data combined from our end for counties and places in the GNRC operating region.  

Go to this page: https://htaindex.cnt.org/download/  
Upon registering for access, download the following documents:  
+ HTA Index for Counties in Tennessee and Kentucky  
+ HTA Index for MPOs  
+ HTA Index for Block Groups in Tennessee  

Save these csvs as they come in the Data Downloads folder of Parent Data Gathering  

### Calculations are made both for Comphrehensive plans at higher geography levels (counties, MPO), but also by block group to identify distressed areas at a granular level.

In [2]:
#Load API Key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
census_key = keys_dict_2['CENSUS']
bea_key = keys_dict_2['BEA']

In [3]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
slaces = ['02180', #Ashland City: Cheatham
          '39660', #Kingston Springs: Cheatham
          '57480', #Pegram: Cheatham
          '59560', #Pleasant View: Cheatham
          '04620', #Belle Meade: Davidson
          '05140', #Berry Hill: Davidson
          '27020', #Forest Hills: Davidson
          '29920', #Goodlettsville: Davidson/Sumner
          '52006', #Nashville-Davidson metropolitan government (balance): Davidson
          '54780', #Oak Hill: Davidson
          '63140', #Ridgetop: Davidson/Robertson
          '09880', #Burns: Dickson
          '13080', #Charlotte: Dickson
          '20620', #Dickson: Dickson
          '69080', #Slayden: Dickson
          '76860', #Vanleer: Dickson
          '79980', #White Bluff: Dickson
          '24320', #Erin: Houston
          '73460', #Tennessee Ridge: Houston/Stewart
          '44840', #McEwen: Humphreys
          '52820', #New Johnsonville: Humphreys
          '78560', #Waverly: Humphreys
          '16540', #Columbia: Maury
          '51080', #Mount Pleasant: Maury
          '70580', #Spring Hill: Maury/Williamson
          '15160', #Clarksville: Montgomery
          '00200', #Adams: Robertson
          '11980', #Cedar Hill: Robertson
          '16980', #Coopertown: Robertson
          '18420', #Cross Plains: Robertson
          '30960', #Greenbrier: Robertson
          '48980', #Millersville: Robertson/Sumner
          '60280', #Portland: Robertson/Sumner
          '70500', #Springfield: Robertson
          '80200', #White House: Robertson/Sumner
          '22360', #Eagleville: Rutherford
          '41200', #La Vergne: Rutherford
          '51560', #Murfreesboro: Rutherford
          '69420', #Smyrna: Rutherford
          '18820', #Cumberland City: Stewart
          '21400', #Dover: Stewart
          '28540', #Gallatin: Sumner
          '33280', #Hendersonville: Sumner
          '79420', #Westmoreland: Sumner
          '08280', #Brentwood: Williamson
          '25440', #Fairview: Williamson
          '27740', #Franklin: Williamson
          '53460', #Nolensville: Williamson
          '73900', #Thompson's Station: Williamson
          '41520', #Lebanon: Wilson
          '50780', #Mount Juliet: Wilson
          '78320', #Watertown: Wilson
          '40160', #Lafayette: Macon
          '69114', #Scottsville city, Kentucky
          '28918', #Franklin city, Kentucky
          '49460'] #Mitchelville city: Sumner

In [4]:
#2019 ACS 5 Year Median Household Income, total occupied housing units for households
#counties
url_str= 'https://api.census.gov/data/2015/acs/acs5?key='+census_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
predicates= {} #block groups GNRC Region
get_vars= ["NAME", 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "block group:*"
predicates["in"]= "state:47, county:*, tract:*"
data= requests.get(url_str, params = predicates)
col_names = ['NAME', 'Median Household Income', 'Households', 'StateFIPS', 'CountyFIPS', 'Census Tract', 'Block Group']
bg=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
bg['GEO_ID'] = bg['StateFIPS'] + bg['CountyFIPS'] + bg['Census Tract'] + bg['Block Group']
bg['GeoFIPS'] = bg['CountyFIPS'] + bg['Census Tract'] + bg['Block Group']
bg = bg.loc[bg['CountyFIPS'].isin(GNRC)]
bg = bg.drop(columns = ['CountyFIPS', 'Census Tract', 'Block Group']).reset_index(drop = True)
df = pd.concat([df, bg], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [5]:
hhincome = savename.reset_index(drop = True)
hhincome = hhincome.drop(columns = ['StateFIPS', 'GeoFIPS'])
hhincome['Median Household Income'] = hhincome['Median Household Income'].astype(float)
hhincome['Households'] = hhincome['Households'].astype(float)

In [6]:
hhincome['Annual Median Household Income'] = hhincome['Median Household Income']
hhincome['Monthly Median Household Income'] = hhincome['Median Household Income']/12
hhincome = hhincome.drop(columns = 'Median Household Income')

In [7]:
hhincome.tail()

,NAME,GEO_ID,Households,Annual Median Household Income,Monthly Median Household Income
1175,"Block Group 4, Census Tract 101.05, Davidson C...",470370101054,322.0,55469.0,4.622417e+03
1176,"Block Group 1, Census Tract 415, Rutherford Co...",471490415001,12.0,-666666666.0,-5.555556e+07
1177,"Block Group 3, Census Tract 418, Rutherford Co...",471490418003,797.0,40168.0,3.347333e+03
1178,"Block Group 4, Census Tract 403.02, Rutherford...",471490403024,754.0,55250.0,4.604167e+03
1179,"Block Group 3, Census Tract 403.02, Rutherford...",471490403023,970.0,53456.0,4.454667e+03


## H&T

In [9]:
kycos = pd.read_csv('../../Data Downloads/htaindex2015_data_counties_21.csv')
tncos = pd.read_csv('../../Data Downloads/htaindex2015_data_counties_47.csv')
#mpos = pd.read_csv('../../Data Downloads/htaindex2015_data_mpos_.csv')
tnplaces = pd.read_csv('../../Data Downloads/htaindex2015_data_places_47.csv')
kyplaces = pd.read_csv('../../Data Downloads/htaindex2015_data_places_21.csv')
tnbg = pd.read_csv('../../Data Downloads/htaindex2015_data_blkgrps_47.csv')

In [10]:
# mpos['name'] = mpos['name'].str.strip('\"')
# mpos['GEO_ID'] = mpos['mpo'].str.strip('\"')
# mpos = mpos.loc[mpos['name'] == 'Nashville Area MPO']

In [11]:
tncos['GEO_ID'] = tncos['county'].str.strip('\"')
kycos['GEO_ID'] = kycos['county'].str.strip('\"')
tnplaces['GEO_ID'] = tnplaces['place'].str.strip('\"')
kyplaces['GEO_ID'] = kyplaces['place'].str.strip('\"')
tnbg['GEO_ID'] = tnbg['blkgrp'].str.strip('\"')

In [12]:
tncos = tncos[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
kycos = kycos[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
tnplaces = tnplaces[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
kyplaces = kyplaces[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
tnbg = tnbg[['GEO_ID', 'h_cost', 't_cost_ami']]
#mpos = mpos[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]

In [13]:
tncos['h_cost_ami'] = tncos['h_cost']*12
kycos['h_cost_ami'] = kycos['h_cost']*12
tnplaces['h_cost_ami'] = tnplaces['h_cost']*12
kyplaces['h_cost_ami'] = kyplaces['h_cost']*12
tnbg['h_cost_ami'] = tnbg['h_cost']*12
#mpos['h_cost_ami'] = mpos['h_cost']*12

In [14]:
tncos= tncos.drop(columns = 'h_cost')
kycos= kycos.drop(columns = 'h_cost')
tnplaces= tnplaces.drop(columns = 'h_cost')
kyplaces= kyplaces.drop(columns = 'h_cost')
tnbg= tnbg.drop(columns = 'h_cost')
#mpos= mpos.drop(columns = 'h_cost')

In [15]:
tncos['GEO_ID'] = tncos['GEO_ID'].str[2:]
kycos['GEO_ID'] = kycos['GEO_ID'].str[2:]
tnplaces['GEO_ID'] = tnplaces['GEO_ID'].str[2:]
kyplaces['GEO_ID'] = kyplaces['GEO_ID'].str[2:]
tnbg['CountyFIPS'] = tnbg['GEO_ID'].str[2:5]

In [16]:
tncos = tncos.loc[tncos['GEO_ID'].isin(GNRC)].reset_index(drop = True)
kycos = kycos.loc[kycos['GEO_ID'].isin(KY)].reset_index(drop = True)
tnplaces = tnplaces.loc[tnplaces['GEO_ID'].isin(slaces)].reset_index(drop = True)
kyplaces = kyplaces.loc[kyplaces['GEO_ID'].isin(slaces)].reset_index(drop = True)
tnbg = tnbg.loc[tnbg['CountyFIPS'].isin(GNRC)]
tnbg = tnbg.drop(columns = ['CountyFIPS']).reset_index(drop = True)

In [17]:
tncos['GEO_ID'] = '0500000US47' + tncos['GEO_ID']
kycos['GEO_ID'] = '0500000US21' + kycos['GEO_ID']
tnplaces['GEO_ID'] = '1600000US47' + tnplaces['GEO_ID']
kyplaces['GEO_ID'] = '1600000US21' + kyplaces['GEO_ID']

In [18]:
df = pd.concat([tncos, kycos, tnplaces, kyplaces, tnbg]).reset_index(drop = True)
data = df.merge(hhincome, on = 'GEO_ID')
data = data.drop(columns = 'name').set_index(['NAME', 'GEO_ID'])

In [19]:
cols = list(data.columns)
data[cols] = data[cols].astype(float)

In [20]:
def percent(x, y):
    return (x/y)*100

In [21]:
data['Annual Median Household Income'] = data['Annual Median Household Income']
data['Monthly Median Household Income'] = data['Annual Median Household Income']/12
data['Annual Transportation Cost'] = data['t_cost_ami']
data['Monthly Transportation Cost'] = data['t_cost_ami']/12
data['Annual Housing Cost'] = data['h_cost_ami']
data['Monthly Housing Cost'] = data['h_cost_ami']/12
data['Annual Housing and Transportation Cost'] = data['Annual Housing Cost'] + data['Annual Transportation Cost']
data['Monthly Housing and Transportation Cost'] = (data['Annual Housing Cost'] + data['Annual Transportation Cost'])/12
data['Transportation Cost as Percent of Monthly Median Household Income'] = percent(data['Monthly Transportation Cost'], data['Monthly Median Household Income'])
data['Transportation Cost as Percent of Annual Median Household Income'] = percent(data['Annual Transportation Cost'], data['Annual Median Household Income'])
data['Housing Cost as Percent of Monthly Median Household Income'] = percent(data['Monthly Housing Cost'], data['Monthly Median Household Income'])
data['Housing Cost as Percent of Annual Median Household Income'] = percent(data['Annual Housing Cost'], data['Annual Median Household Income'])
data['Housing and Transportation Cost as Percent of Monthly Median Household Income'] = percent(data['Monthly Housing and Transportation Cost'], data['Monthly Median Household Income'])
data['Housing and Transportation Cost as Percent of Annual Median Household Income'] = percent(data['Annual Housing and Transportation Cost'], data['Annual Median Household Income'])

In [22]:
data = data.drop(columns = ['t_cost_ami', 'h_cost_ami'])

In [23]:
data = data.reset_index()

In [24]:
data['Difference Annual Median Household Income and Total Annual Costs'] = data['Annual Median Household Income'] - data['Annual Housing and Transportation Cost']

In [25]:
data.sort_values('Difference Annual Median Household Income and Total Annual Costs', ascending = True).reset_index(drop = True)

,NAME,GEO_ID,Households,Annual Median Household Income,Monthly Median Household Income,Annual Transportation Cost,Monthly Transportation Cost,Annual Housing Cost,Monthly Housing Cost,Annual Housing and Transportation Cost,Monthly Housing and Transportation Cost,Transportation Cost as Percent of Monthly Median Household Income,Transportation Cost as Percent of Annual Median Household Income,Housing Cost as Percent of Monthly Median Household Income,Housing Cost as Percent of Annual Median Household Income,Housing and Transportation Cost as Percent of Monthly Median Household Income,Housing and Transportation Cost as Percent of Annual Median Household Income,Difference Annual Median Household Income and Total Annual Costs
0,"Block Group 4, Census Tract 162, Davidson Coun...",470370162004,313.0,15781.0,1.315083e+03,10169.0,847.416667,15336.0,1278.0,25505.0,2125.416667,64.438249,64.438249,97.180153,97.180153,161.618402,161.618402,-9724.0
1,"Block Group 2, Census Tract 162, Davidson Coun...",470370162002,402.0,12385.0,1.032083e+03,9946.0,828.833333,9720.0,810.0,19666.0,1638.833333,80.306823,80.306823,78.482035,78.482035,158.788857,158.788857,-7281.0
2,"Block Group 1, Census Tract 148, Davidson Coun...",470370148001,166.0,18750.0,1.562500e+03,12197.0,1016.416667,12852.0,1071.0,25049.0,2087.416667,65.050667,65.050667,68.544000,68.544000,133.594667,133.594667,-6299.0
3,"Block Group 4, Census Tract 128.01, Davidson C...",470370128014,780.0,14643.0,1.220250e+03,11110.0,925.833333,9384.0,782.0,20494.0,1707.833333,75.872431,75.872431,64.085228,64.085228,139.957659,139.957659,-5851.0
4,"Cumberland City town, Tennessee",1600000US4718820,142.0,18750.0,1.562500e+03,13925.0,1160.416667,10524.0,877.0,24449.0,2037.416667,74.266667,74.266667,56.128000,56.128000,130.394667,130.394667,-5699.0
5,"Block Group 2, Census Tract 418, Rutherford Co...",471490418002,417.0,12270.0,1.022500e+03,11183.0,931.916667,6540.0,545.0,17723.0,1476.916667,91.140994,91.140994,53.300733,53.300733,144.441728,144.441728,-5453.0
6,"Block Group 1, Census Tract 110.01, Davidson C...",470370110011,447.0,15168.0,1.264000e+03,12212.0,1017.666667,8064.0,672.0,20276.0,1689.666667,80.511603,80.511603,53.164557,53.164557,133.676160,133.676160,-5108.0
7,"Block Group 3, Census Tract 194, Davidson Coun...",470370194003,505.0,9002.0,7.501667e+02,9673.0,806.083333,3996.0,333.0,13669.0,1139.083333,107.453899,107.453899,44.390136,44.390136,151.844035,151.844035,-4667.0
8,"Block Group 3, Census Tract 419, Rutherford Co...",471490419003,575.0,16641.0,1.386750e+03,11553.0,962.750000,9516.0,793.0,21069.0,1755.750000,69.424914,69.424914,57.184063,57.184063,126.608978,126.608978,-4428.0
9,"Block Group 1, Census Tract 307, Wilson County...",471890307001,496.0,16292.0,1.357667e+03,12059.0,1004.916667,8556.0,713.0,20615.0,1717.916667,74.017923,74.017923,52.516573,52.516573,126.534495,126.534495,-4323.0


In [26]:
data = data.set_index(['NAME', 'GEO_ID'])
data.head()

,,Households,Annual Median Household Income,Monthly Median Household Income,Annual Transportation Cost,Monthly Transportation Cost,Annual Housing Cost,Monthly Housing Cost,Annual Housing and Transportation Cost,Monthly Housing and Transportation Cost,Transportation Cost as Percent of Monthly Median Household Income,Transportation Cost as Percent of Annual Median Household Income,Housing Cost as Percent of Monthly Median Household Income,Housing Cost as Percent of Annual Median Household Income,Housing and Transportation Cost as Percent of Monthly Median Household Income,Housing and Transportation Cost as Percent of Annual Median Household Income,Difference Annual Median Household Income and Total Annual Costs
NAME,GEO_ID,,,,,,,,,,,,,,,,
"Cheatham County, Tennessee",0500000US47021,14499.0,51857.0,4321.416667,14845.0,1237.083333,13980.0,1165.0,28825.0,2402.083333,28.626801,28.626801,26.958752,26.958752,55.585553,55.585553,23032.0
"Davidson County, Tennessee",0500000US47037,264211.0,48368.0,4030.666667,12082.0,1006.833333,14508.0,1209.0,26590.0,2215.833333,24.979325,24.979325,29.995038,29.995038,54.974363,54.974363,21778.0
"Dickson County, Tennessee",0500000US47043,18556.0,44680.0,3723.333333,14503.0,1208.583333,12288.0,1024.0,26791.0,2232.583333,32.459714,32.459714,27.502238,27.502238,59.961952,59.961952,17889.0
"Houston County, Tennessee",0500000US47083,3247.0,39401.0,3283.416667,13114.0,1092.833333,9564.0,797.0,22678.0,1889.833333,33.283419,33.283419,24.273496,24.273496,57.556915,57.556915,16723.0
"Humphreys County, Tennessee",0500000US47085,7124.0,41949.0,3495.750000,13919.0,1159.916667,11088.0,924.0,25007.0,2083.916667,33.180767,33.180767,26.432096,26.432096,59.612863,59.612863,16942.0


In [30]:
data = data.add_suffix(' 2015')

In [36]:
data = data.reset_index()
data.head()

,NAME,GEO_ID,Households 2015,Annual Median Household Income 2015,Monthly Median Household Income 2015,Annual Transportation Cost 2015,Monthly Transportation Cost 2015,Annual Housing Cost 2015,Monthly Housing Cost 2015,Annual Housing and Transportation Cost 2015,Monthly Housing and Transportation Cost 2015,Transportation Cost as Percent of Monthly Median Household Income 2015,Transportation Cost as Percent of Annual Median Household Income 2015,Housing Cost as Percent of Monthly Median Household Income 2015,Housing Cost as Percent of Annual Median Household Income 2015,Housing and Transportation Cost as Percent of Monthly Median Household Income 2015,Housing and Transportation Cost as Percent of Annual Median Household Income 2015,Difference Annual Median Household Income and Total Annual Costs 2015
0,"Cheatham County, Tennessee",0500000US47021,14499.0,51857.0,4321.416667,14845.0,1237.083333,13980.0,1165.0,28825.0,2402.083333,28.626801,28.626801,26.958752,26.958752,55.585553,55.585553,23032.0
1,"Davidson County, Tennessee",0500000US47037,264211.0,48368.0,4030.666667,12082.0,1006.833333,14508.0,1209.0,26590.0,2215.833333,24.979325,24.979325,29.995038,29.995038,54.974363,54.974363,21778.0
2,"Dickson County, Tennessee",0500000US47043,18556.0,44680.0,3723.333333,14503.0,1208.583333,12288.0,1024.0,26791.0,2232.583333,32.459714,32.459714,27.502238,27.502238,59.961952,59.961952,17889.0
3,"Houston County, Tennessee",0500000US47083,3247.0,39401.0,3283.416667,13114.0,1092.833333,9564.0,797.0,22678.0,1889.833333,33.283419,33.283419,24.273496,24.273496,57.556915,57.556915,16723.0
4,"Humphreys County, Tennessee",0500000US47085,7124.0,41949.0,3495.750000,13919.0,1159.916667,11088.0,924.0,25007.0,2083.916667,33.180767,33.180767,26.432096,26.432096,59.612863,59.612863,16942.0


In [33]:
data.to_csv('../../Outputs/CNT_HT_2015.csv', index = False)

In [37]:
new = pd.read_csv('../../Outputs/CNT_HT_2019.csv')

In [38]:
df = data.merge(new)

In [40]:
def percentchange(x, y):
    return ((x-y)/y)*100

In [39]:
df.head()

,NAME,GEO_ID,Households 2015,Annual Median Household Income 2015,Monthly Median Household Income 2015,Annual Transportation Cost 2015,Monthly Transportation Cost 2015,Annual Housing Cost 2015,Monthly Housing Cost 2015,Annual Housing and Transportation Cost 2015,Monthly Housing and Transportation Cost 2015,Transportation Cost as Percent of Monthly Median Household Income 2015,Transportation Cost as Percent of Annual Median Household Income 2015,Housing Cost as Percent of Monthly Median Household Income 2015,Housing Cost as Percent of Annual Median Household Income 2015,Housing and Transportation Cost as Percent of Monthly Median Household Income 2015,Housing and Transportation Cost as Percent of Annual Median Household Income 2015,Difference Annual Median Household Income and Total Annual Costs 2015,Households 2019,Annual Median Household Income 2019,Monthly Median Household Income 2019,Annual Transportation Cost 2019,Monthly Transportation Cost 2019,Annual Housing Cost 2019,Monthly Housing Cost 2019,Annual Housing and Transportation Cost 2019,Monthly Housing and Transportation Cost 2019,Transportation Cost as Percent of Monthly Median Household Income 2019,Transportation Cost as Percent of Annual Median Household Income 2019,Housing Cost as Percent of Monthly Median Household Income 2019,Housing Cost as Percent of Annual Median Household Income 2019,Housing and Transportation Cost as Percent of Monthly Median Household Income 2019,Housing and Transportation Cost as Percent of Annual Median Household Income 2019,Difference Annual Median Household Income and Total Annual Costs 2019
0,"Cheatham County, Tennessee",0500000US47021,14499.0,51857.0,4321.416667,14845.0,1237.083333,13980.0,1165.0,28825.0,2402.083333,28.626801,28.626801,26.958752,26.958752,55.585553,55.585553,23032.0,15089.0,61913.0,5159.416667,16511.0,1375.916667,14748.0,1229.0,31259.0,2604.916667,26.668066,26.668066,23.820522,23.820522,50.488589,50.488589,30654.0
1,"Davidson County, Tennessee",0500000US47037,264211.0,48368.0,4030.666667,12082.0,1006.833333,14508.0,1209.0,26590.0,2215.833333,24.979325,24.979325,29.995038,29.995038,54.974363,54.974363,21778.0,282366.0,60388.0,5032.333333,13136.0,1094.666667,16308.0,1359.0,29444.0,2453.666667,21.752666,21.752666,27.005365,27.005365,48.758031,48.758031,30944.0
2,"Dickson County, Tennessee",0500000US47043,18556.0,44680.0,3723.333333,14503.0,1208.583333,12288.0,1024.0,26791.0,2232.583333,32.459714,32.459714,27.502238,27.502238,59.961952,59.961952,17889.0,19198.0,53076.0,4423.000000,16270.0,1355.833333,13092.0,1091.0,29362.0,2446.833333,30.654156,30.654156,24.666516,24.666516,55.320672,55.320672,23714.0
3,"Houston County, Tennessee",0500000US47083,3247.0,39401.0,3283.416667,13114.0,1092.833333,9564.0,797.0,22678.0,1889.833333,33.283419,33.283419,24.273496,24.273496,57.556915,57.556915,16723.0,2878.0,42711.0,3559.250000,15210.0,1267.500000,11364.0,947.0,26574.0,2214.500000,35.611435,35.611435,26.606729,26.606729,62.218164,62.218164,16137.0
4,"Humphreys County, Tennessee",0500000US47085,7124.0,41949.0,3495.750000,13919.0,1159.916667,11088.0,924.0,25007.0,2083.916667,33.180767,33.180767,26.432096,26.432096,59.612863,59.612863,16942.0,6763.0,45667.0,3805.583333,15296.0,1274.666667,10764.0,897.0,26060.0,2171.666667,33.494646,33.494646,23.570631,23.570631,57.065277,57.065277,19607.0


In [43]:
df['Change in Annual MHI'] = df['Annual Median Household Income 2019'] - df['Annual Median Household Income 2015']
df['Percent Change in Annual MHI'] = percentchange(df['Annual Median Household Income 2019'], df['Annual Median Household Income 2015'])
df['Change in Monthly MHI'] = df['Monthly Median Household Income 2019'] - df['Monthly Median Household Income 2015']
df['Percent Change in Monthly MHI'] = percentchange(df['Monthly Median Household Income 2019'], df['Monthly Median Household Income 2015'])
df['Change in Annual Transportation Cost'] = df['Annual Transportation Cost 2019'] - df['Annual Transportation Cost 2015']
df['Percent Change in Annual Transportation Cost'] = percentchange(df['Annual Transportation Cost 2019'], df['Annual Transportation Cost 2015'])
df['Change in Monthly Transportation Cost'] = df['Monthly Transportation Cost 2019'] - df['Monthly Transportation Cost 2015']
df['Percent Change in Monthly Transportation Cost'] = percentchange(df['Monthly Transportation Cost 2019'], df['Monthly Transportation Cost 2015'])
df['Change in Annual Transportation Cost'] = df['Annual Transportation Cost 2019'] - df['Annual Transportation Cost 2015']
df['Percent Change in Annual Transportation Cost'] = percentchange(df['Annual Transportation Cost 2019'], df['Annual Transportation Cost 2015'])
df['Change in Monthly Transportation Cost'] = df['Monthly Transportation Cost 2019'] - df['Monthly Transportation Cost 2015']
df['Percent Change in Monthly Transportation Cost'] = percentchange(df['Monthly Transportation Cost 2019'], df['Monthly Transportation Cost 2015'])
df['Change in Annual Housing and Transportation Cost'] = df['Annual Housing and Transportation Cost 2019'] - df['Annual Housing and Transportation Cost 2015']
df['Percent Change in Annual Housing and Transportation Cost'] = percentchange(df['Annual Housing and Transportation Cost 2019'], 
                                                                    df['Annual Housing and Transportation Cost 2015'])
df['Change in Monthly Housing and Transportation Cost'] = df['Monthly Housing and Transportation Cost 2019'] - df['Monthly Housing and Transportation Cost 2015']
df['Percent Change in Monthly Housing and Transportation Cost'] = percentchange(df['Monthly Housing and Transportation Cost 2019'], 
                                                                    df['Monthly Housing and Transportation Cost 2015'])
df['Change in Difference Annual Median Household Income and Total Annual Costs'] = df['Difference Annual Median Household Income and Total Annual Costs 2019'] - df['Difference Annual Median Household Income and Total Annual Costs 2015']
df['Percent Change in Difference Annual Median Household Income and Total Annual Costs'] = percentchange(df['Difference Annual Median Household Income and Total Annual Costs 2019'], 
                                                                    df['Difference Annual Median Household Income and Total Annual Costs 2015'])

In [44]:
df.head()

,NAME,GEO_ID,Households 2015,Annual Median Household Income 2015,Monthly Median Household Income 2015,Annual Transportation Cost 2015,Monthly Transportation Cost 2015,Annual Housing Cost 2015,Monthly Housing Cost 2015,Annual Housing and Transportation Cost 2015,Monthly Housing and Transportation Cost 2015,Transportation Cost as Percent of Monthly Median Household Income 2015,Transportation Cost as Percent of Annual Median Household Income 2015,Housing Cost as Percent of Monthly Median Household Income 2015,Housing Cost as Percent of Annual Median Household Income 2015,Housing and Transportation Cost as Percent of Monthly Median Household Income 2015,Housing and Transportation Cost as Percent of Annual Median Household Income 2015,Difference Annual Median Household Income and Total Annual Costs 2015,Households 2019,Annual Median Household Income 2019,Monthly Median Household Income 2019,Annual Transportation Cost 2019,Monthly Transportation Cost 2019,Annual Housing Cost 2019,Monthly Housing Cost 2019,Annual Housing and Transportation Cost 2019,Monthly Housing and Transportation Cost 2019,Transportation Cost as Percent of Monthly Median Household Income 2019,Transportation Cost as Percent of Annual Median Household Income 2019,Housing Cost as Percent of Monthly Median Household Income 2019,Housing Cost as Percent of Annual Median Household Income 2019,Housing and Transportation Cost as Percent of Monthly Median Household Income 2019,Housing and Transportation Cost as Percent of Annual Median Household Income 2019,Difference Annual Median Household Income and Total Annual Costs 2019,Change in Annual MHI,Percent Change in Annual MHI,Change in Monthly MHI,Percent Change in Monthly MHI,Change in Annual Transportation Cost,Percent Change in Annual Transportation Cost,Change in Monthly Transportation Cost,Percent Change in Monthly Transportation Cost,Change in Annual Housing and Transportation Cost,Percent Change in Annual Housing and Transportation Cost,Change in Monthly Housing and Transportation Cost,Percent Change in Monthly Housing and Transportation Cost,Change in Difference Annual Median Household Income and Total Annual Costs,Percent Change in Difference Annual Median Household Income and Total Annual Costs
0,"Cheatham County, Tennessee",0500000US47021,14499.0,51857.0,4321.416667,14845.0,1237.083333,13980.0,1165.0,28825.0,2402.083333,28.626801,28.626801,26.958752,26.958752,55.585553,55.585553,23032.0,15089.0,61913.0,5159.416667,16511.0,1375.916667,14748.0,1229.0,31259.0,2604.916667,26.668066,26.668066,23.820522,23.820522,50.488589,50.488589,30654.0,10056.0,19.391789,838.000000,19.391789,1666.0,11.222634,138.833333,11.222634,2434.0,8.444059,202.833333,8.444059,7622.0,33.093088
1,"Davidson County, Tennessee",0500000US47037,264211.0,48368.0,4030.666667,12082.0,1006.833333,14508.0,1209.0,26590.0,2215.833333,24.979325,24.979325,29.995038,29.995038,54.974363,54.974363,21778.0,282366.0,60388.0,5032.333333,13136.0,1094.666667,16308.0,1359.0,29444.0,2453.666667,21.752666,21.752666,27.005365,27.005365,48.758031,48.758031,30944.0,12020.0,24.851141,1001.666667,24.851141,1054.0,8.723721,87.833333,8.723721,2854.0,10.733358,237.833333,10.733358,9166.0,42.088346
2,"Dickson County, Tennessee",0500000US47043,18556.0,44680.0,3723.333333,14503.0,1208.583333,12288.0,1024.0,26791.0,2232.583333,32.459714,32.459714,27.502238,27.502238,59.961952,59.961952,17889.0,19198.0,53076.0,4423.000000,16270.0,1355.833333,13092.0,1091.0,29362.0,2446.833333,30.654156,30.654156,24.666516,24.666516,55.320672,55.320672,23714.0,8396.0,18.791406,699.666667,18.791406,1767.0,12.183686,147.250000,12.183686,2571.0,9.596506,214.250000,9.596506,5825.0,32.561910
3,"Houston County, Tennessee",0500000US47083,3247.0,39401.0,3283.416667,13114.0,1092.833333,9564.0,797.0,22678.0,1889.833333,33.283419,33.283419,24.273496,24.273496,57.556915,57.556915,16723.0,2878.0,42711.0,3559.250000,15210.0,1267.500000,11364.0,947.0,26574.0,2214.500000,35.611435,35.611435,26.606729,26.606729,62.21

In [46]:
df.to_csv('../../Outputs/CNT_HT_OverTime.csv', index = False)